In [1]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta

In [6]:
def bulldog_page_job_offers(
    url="https://bulldogjob.pl/companies/jobs/s/city,Warszawa,Remote/skills,Python/experienceLevel,junior",
) -> list:

    bulldog_list = list()

    flag = True
    try:
        page = requests.get(url)
    except requests.exceptions.ConnectionError as err:
        flag = False
    except requests.exceptions.MissingSchema as err:
        flag = False
    if flag:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find(id="__next")
        job_elements = results.find_all(
            "div",
            class_="py-6 md:py-8 px-8 flex flex-wrap relative bg-white mb-2 rounded-lg shadow",
        )

        for job_element in job_elements:
            (
                publication_date,
                company,
                job_title,
                position,
                website_name,
                link_url,
            ) = get_bulldog_job_details(job_element)
            bulldog_dict = dict_creator(
                publication_date, company, job_title, position, website_name, link_url
            )
            bulldog_list.append(bulldog_dict)
    return bulldog_list


In [12]:
def get_bulldog_job_details(job_element):
    job_title_element = job_element.find(
        "h3", class_="text-c28 font-medium mb-3 w-full md:hidden"
    )
    job_title = job_title_element.text.strip()

    position_element = job_element.find("p", class_="tracking-05 uppercase md:my-4")
    position = position_element.text.strip().capitalize()

    company_element = job_element.find(
        "p",
        class_="text-sm md:text-xxs md:text-center my-2 font-medium text-gray-300",
    )
    company = company_element.text.strip()

    links = job_element.find("h3", class_="text-c28 font-medium mb-3 w-full md:hidden")

    for link in links:
        link_url = link["href"]
    pattern = re.compile(r"https?://([\w.\.\-]+)")
    website_name = pattern.match(link_url)[0]
    page_job_element = requests.get(link_url)
    soup_page = BeautifulSoup(page_job_element.content, "html.parser")
    publication_date_element = soup_page.find(
        "p", class_="text-md xl:text-c22 leading-6"
    )
    days_after_publication = int(
        re.findall(r"\b\d+\b", publication_date_element.text.strip())[0]
    )
    print(days_after_publication)
    publication_date = (
        datetime.today() - (timedelta(days=days_after_publication))
    ).strftime("%Y-%m-%d")

    return (publication_date, company, job_title, position, website_name, link_url)

In [10]:
def dict_creator(
    publication_date, company, job_title, position, website_name, link_url
):
    offers_dict = dict()
    offers_dict["publication_date"] = publication_date
    offers_dict["company"] = company
    offers_dict["title"] = job_title
    offers_dict["position"] = position
    offers_dict["website"] = website_name
    offers_dict["link_url"] = link_url

    return offers_dict

In [13]:
import pandas as pd
import numpy as np

bulldog_list = bulldog_page_job_offers()
df_raw = pd.DataFrame.from_records(bulldog_list)
df = df_raw.copy()
df['publication_date'] = pd.to_datetime(df['publication_date'], infer_datetime_format=True)
df.drop_duplicates(subset=['publication_date', 'company', 'title'], inplace=True, ignore_index=True)

6
6


IndexError: list index out of range

In [5]:
df

,publication_date,company,title,position,link_url,website
0,2022-10-18,Accenture Polska,Junior Data Engineer with Python (Communicatio...,Junior,https://bulldogjob.pl/companies/jobs/127078-ju...,https://bulldogjob.pl
